In [21]:
from langchain import OpenAI

In [22]:
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

True

In [23]:

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY") 
HUGGINGFACEHUB_API_TOKEN = os.environ.get("HUGGINGFACEHUB_API_TOKEN")


In [27]:
llm = OpenAI(
    openai_api_key = os.environ["OPENAI_API_KEY"],
    temperature = 0.6
)

In [28]:
text = "What is the H2O"

print(llm.predict(text))

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-hOwgY***************************************lNZp. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
# HUGGINGFACEHUB_API_TOKEN = os.environ.get("HUGGINGFACEHUB_API_TOKEN")

# os.environ["HUGGINGFACEHUB_API_TOKEN"]

In [2]:
from langchain import HuggingFaceHub
llm_hf = HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={"temperature":0,"max_length":512})

f:\FellowshipAI\LangChain\LangChainScratch\venv\lib\site-packages\huggingface_hub\utils\_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [3]:
output=llm_hf.predict("Can you tell me the capital of Bangladesh")

print(output)

Dhaka


In [5]:
output=llm_hf.predict("What is a pizzas main ingredient?")

print(output)

tomatoes


## Prompt Template

In [10]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(
    input_variables = ["country"],
    template = "Tell me the capital of this {country}"
)

prompt_template.format(country= "Finland")

'Tell me the capital of this Finland'

In [19]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm_hf, prompt=prompt_template)
print(chain.run("Bangladesh"))

dhaka


### Combining Multiple Chains Using Sequential Chain

In [29]:
capital_template=PromptTemplate(
    input_variables=['country'],
    template="Please tell me the capital of the {country}")

capital_chain=LLMChain(llm=llm_hf,prompt=capital_template)

famous_template=PromptTemplate(
    input_variables=['capital'],
    template="Suggest me some amazing places to visit in {capital}")

famous_chain=LLMChain(llm=llm_hf,prompt=famous_template)


In [32]:
from langchain.chains import SimpleSequentialChain
chain=SimpleSequentialChain(chains=[capital_chain,famous_chain])
chain.run("Bangladesh")

'The National Museum of Bangladesh'

### Sequential Chain

In [33]:
capital_template=PromptTemplate(
    input_variables=['country'],
    template="Please tell me the capital of the {country}")

capital_chain=LLMChain(llm=llm_hf,prompt=capital_template,output_key="capital")

In [34]:
famous_template=PromptTemplate(
    input_variables=['capital'],
    template="Suggest me some amazing places to visit in {capital}")

famous_chain=LLMChain(llm=llm_hf,prompt=famous_template,output_key="places")

In [35]:
from langchain.chains import SequentialChain
chain=SequentialChain(chains=[capital_chain,famous_chain],
input_variables=['country'],
output_variables=['capital',"places"])

In [36]:
chain({'country':"Bangladesh"})

{'country': 'Bangladesh',
 'capital': 'dhaka',
 'places': 'The National Museum of Bangladesh'}